In [28]:
import json
import re
from collections import defaultdict
from tqdm import tqdm
import os, random, numpy as np, torch
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from torch_geometric.data import Data
from collections import defaultdict
import json
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
from sklearn.metrics import f1_score, accuracy_score
import json, re, numpy as np
from collections import defaultdict
from tqdm import tqdm
import torch, torch.nn as nn
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATv2Conv
from transformers import BertTokenizer, BertModel
from sklearn.metrics import f1_score, accuracy_score

SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True, warn_only=True)

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(SEED)

def clean_message(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)  
    text = re.sub(r'[^a-z0-9\s]', '', text)            
    text = re.sub(r'\s+', ' ', text).strip()           
    return text

def add_history_and_score_delta(input_path, output_path, k=5):
    with open(input_path, 'r') as f_in, open(output_path, 'w') as f_out:
        for line in tqdm(f_in, desc="Processing games"):
            if not line.strip():
                continue

            game = json.loads(line)
            game_id = game.get('game_id', 'UNKNOWN')

            sorted_indices = sorted(range(len(game['messages'])),
                                    key=lambda i: game.get("absolute_message_index", list(range(len(game['messages']))))[i])

            game['messages'] = [game['messages'][i] for i in sorted_indices]
            game['speakers'] = [game['speakers'][i] for i in sorted_indices]
            game['receivers'] = [game['receivers'][i] for i in sorted_indices]
            game['seasons'] = [game['seasons'][i] for i in sorted_indices]
            game['years'] = [game['years'][i] for i in sorted_indices]
            if 'absolute_message_index' in game:
                game['absolute_message_index'] = [game['absolute_message_index'][i] for i in sorted_indices]
            if 'game_score_delta' in game:
                game['score_delta'] = [float(game['game_score_delta'][i]) for i in sorted_indices]
            else:
                game['score_delta'] = [0.0] * len(game['messages'])

            history_lookup = defaultdict(list)
            game['history'] = []
            for i in range(len(game['messages'])):
                sender = game['speakers'][i]
                receiver = game['receivers'][i]
                pair_key = (game_id, sender, receiver)

                cleaned = clean_message(game['messages'][i])
                game['messages'][i] = cleaned

                last_k = history_lookup[pair_key][-k:]
                game['history'].append(last_k)

                history_lookup[pair_key].append(cleaned)

            f_out.write(json.dumps(game) + "\n")


add_history_and_score_delta("/kaggle/input/deception/train.jsonl", "train_with_history_10.jsonl", k=10)
add_history_and_score_delta("/kaggle/input/deception/validation.jsonl", "val_with_history_10.jsonl", k=10)
add_history_and_score_delta("/kaggle/input/deception/test.jsonl", "test_with_history_10.jsonl", k=10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
COUNTRY2IDX = {"russia":0, "turkey":1, "england":2, "france":3,
               "germany":4, "italy":5, "austria":6}
SEASON2IDX  = {"Spring":0, "Fall":1, "Winter":2}
YEAR_BUCKETS = [1901, 1906, 1911, 1916, 1921]

def clean_msg(txt: str) -> str:
    txt = txt.lower()
    txt = re.sub(r'https?://\S+|www\.\S+', '', txt)
    txt = re.sub(r'[^a-z0-9\s]', ' ', txt)
    return re.sub(r'\s+', ' ', txt).strip()

def year_bucket(y: int) -> int:
    for i, b in enumerate(YEAR_BUCKETS):
        if y < b:
            return i
    return len(YEAR_BUCKETS) - 1

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert      = BertModel.from_pretrained("bert-base-uncased").eval().to(device)

@torch.inference_mode()
def bert_cls(text: str) -> torch.Tensor:
    ids = tokenizer(text, return_tensors="pt", truncation=True,
                    padding=True).to(device)
    return bert(**ids).last_hidden_state[:, 0, :].squeeze(0).cpu()   # [768]

LEX_PATH = "/kaggle/input/deception-lexicon/2015_Diplomacy_lexicon.json"
with open(LEX_PATH) as f:
    _lex = json.load(f)

_lex["but"] = ["but"]
_lex["countries"] = ["austria","england","france","germany","italy","russia","turkey"]

LEX_KEYS = list(_lex.keys())
LEX_CUES = {k:[c.lower() for c in _lex[k]] for k in LEX_KEYS}

try:
    import spacy, warnings
    warnings.filterwarnings("ignore", category=UserWarning)
    _nlp = spacy.blank("en")
    tokenize = lambda txt:[t.text for t in _nlp(txt)]
except Exception:
    tokenize = lambda txt: re.findall(r"\b\w+\b", txt.lower())

def count_lexicon(msg:str)->np.ndarray:
    msg_l = msg.lower()
    tokens = set(tokenize(msg_l))
    counts=[]
    for k in LEX_KEYS:
        total=0
        for cue in LEX_CUES[k]:
            total += cue in msg_l if ' ' in cue else cue in tokens
        counts.append(total)
    return np.asarray(counts,dtype=np.float32)   # len==10




def build_game_graph(game_json) -> Data:
    players = sorted(set(map(str.lower,
                         game_json["speakers"] + game_json["receivers"])))
    n = len(players)

    x = torch.zeros(n, len(COUNTRY2IDX))         
    for i, p in enumerate(players):
        x[i, COUNTRY2IDX.get(p, 0)] = 1.0

    lie_running      = defaultdict(int)   
    delta_sum        = defaultdict(float) 
    delta_cnt        = defaultdict(int)   

    e_src, e_dst, e_attr, e_y = [], [], [], []

    abs_idx = game_json.get("absolute_message_index", [])
    max_abs = max(abs_idx) if abs_idx else 1

    for i, raw in enumerate(game_json["messages"]):
        s = game_json["speakers"][i].lower()
        r = game_json["receivers"][i].lower()
        if s not in players or r not in players:
            continue
        s_idx, r_idx = players.index(s), players.index(r)

        clean     = clean_msg(raw)
        bert_vec  = bert_cls(clean)
        lex_vec   = torch.from_numpy(count_lexicon(clean))

        lie_so_far = lie_running[s]             
        lie_feat   = torch.tensor([lie_so_far], dtype=torch.float)

        if delta_cnt[s] == 0:
            avg_delta = 0.0                      
        else:
            avg_delta = delta_sum[s] / delta_cnt[s]
        power_feat = torch.tensor([avg_delta], dtype=torch.float)

        meta = torch.tensor([
            SEASON2IDX[game_json["seasons"][i]],
            year_bucket(int(game_json["years"][i])),
            game_json["absolute_message_index"][i] / max_abs
        ])

        e_attr.append(torch.cat([bert_vec,
                                 lex_vec,
                                 lie_feat,
                                 power_feat,
                                 meta]))
        e_src.append(s_idx)
        e_dst.append(r_idx)

        label = 0 if game_json["sender_labels"][i] in (True, "true") else 1
        e_y.append(label)

        lie_running[s] += label
        delta          = float(game_json["score_delta"][i])  
        delta_sum[s]  += delta
        delta_cnt[s]  += 1

    if not e_attr:                    
        return None

    edge_index = torch.tensor([e_src, e_dst])
    edge_attr  = torch.stack(e_attr)
    edge_label = torch.tensor(e_y, dtype=torch.float)

    return Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr,
        edge_label=edge_label
    )

class DiplomacyGraphDataset(InMemoryDataset):
    def __init__(self,path:str):
        super().__init__(".")
        with open(path) as f:
            data=[build_game_graph(json.loads(l))
                  for l in f if l.strip()]
        data=[d for d in data if d is not None]   
        self.data,self.slices=self.collate(data)

class LieDetectorGAT(nn.Module):
    def __init__(self,node_in,edge_in,hid=256,heads=4,layers=3):
        super().__init__()
        self.edge_enc=nn.Sequential(nn.Linear(edge_in,hid),nn.ReLU(),
                                    nn.Linear(hid,hid))
        self.node_proj=nn.Linear(node_in,hid)
        self.gats=nn.ModuleList([GATv2Conv(hid,hid//heads,heads=heads,
                                           edge_dim=hid,add_self_loops=False)
                                 for _ in range(layers)])
        self.classifier=nn.Sequential(
            nn.Linear(hid*2+hid,256),nn.ReLU(),
            nn.Dropout(0.3),nn.Linear(256,1))

    def forward(self,data):
        x=self.node_proj(data.x)
        ea=self.edge_enc(data.edge_attr)
        for gat in self.gats:
            x=gat(x,data.edge_index,ea).relu()
        h_s,h_r=x[data.edge_index[0]],x[data.edge_index[1]]
        return self.classifier(torch.cat([h_s,h_r,ea],1)).view(-1)


train_ds=DiplomacyGraphDataset("train_with_history_10.jsonl")
val_ds  =DiplomacyGraphDataset("val_with_history_10.jsonl")

train_loader=DataLoader(train_ds,batch_size=1)
val_loader  =DataLoader(val_ds,batch_size=1)


model=LieDetectorGAT(train_ds[0].x.size(1),train_ds[0].edge_attr.size(1)).to(device)

num_true,num_false=12541,591
criterion=nn.BCEWithLogitsLoss(
    pos_weight=torch.tensor([num_true/num_false],device=device))
opt=torch.optim.AdamW(model.parameters(),lr=3e-4)

@torch.no_grad()
def evaluate(loader):
    model.eval(); preds,gold=[],[]
    for data in loader:
        data=data.to(device)
        if data.edge_label.numel()==0:   
            continue
        prob=torch.sigmoid(model(data))
        preds+=(prob>0.5).long().cpu().tolist()
        gold +=data.edge_label.long().cpu().tolist()
    f1  = f1_score(gold,preds,average="macro")
    lie = f1_score(gold,preds,pos_label=1,average="binary")
    acc = accuracy_score(gold,preds)
    return f1,lie,acc

best_val_f1 = 0.0
for epoch in range(20):
    model.train(); running = 0.0
    for data in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        if data.edge_label.numel() == 0:   
            continue
        data = data.to(device)
        opt.zero_grad()
        loss = criterion(model(data), data.edge_label.to(device))
        loss.backward(); nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        opt.step(); running += loss.item()

    tr_f1, tr_lie, tr_acc = evaluate(train_loader)
    va_f1, va_lie, va_acc = evaluate(val_loader)

    print(f"\nEpoch {epoch+1:02d} | "
          f"Loss {running/len(train_loader):.4f} || "
          f"Train F1 {tr_f1:.4f} Acc {tr_acc:.4f} LieF1 {tr_lie:.4f} || "
          f"Val F1 {va_f1:.4f} Acc {va_acc:.4f} LieF1 {va_lie:.4f} || "
          )

    if va_f1 > best_val_f1:
        best_val_f1 = va_f1
        torch.save(model.state_dict(), "GATmodel.pt")
        print(f"  ====> Saving new model with best Val F1 (Val F1: {va_f1:.4f}) <====")

Processing games: 189it [00:00, 547.22it/s]
Processing games: 21it [00:00, 644.29it/s]
Processing games: 42it [00:00, 652.76it/s]
Epoch 1: 100%|██████████| 184/184 [00:02<00:00, 64.39it/s]



Epoch 01 | Loss 2.6940 || Train F1 0.5196 Acc 0.9544 LieF1 0.0626 || Val F1 0.4899 Acc 0.9605 LieF1 0.0000 || 
  ====> Saving new model with best Val F1 (Val F1: 0.4899) <====


Epoch 2: 100%|██████████| 184/184 [00:02<00:00, 64.84it/s]



Epoch 02 | Loss 2.5604 || Train F1 0.6026 Acc 0.9245 LieF1 0.2451 || Val F1 0.5356 Acc 0.9386 LieF1 0.1031 || 
  ====> Saving new model with best Val F1 (Val F1: 0.5356) <====


Epoch 3: 100%|██████████| 184/184 [00:03<00:00, 59.70it/s]


...


Epoch 18 | Loss 1.2888 || Train F1 0.6988 Acc 0.9266 LieF1 0.4369 || Val F1 0.5341 Acc 0.8919 LieF1 0.1257 || 


Epoch 19: 100%|██████████| 184/184 [00:02<00:00, 64.69it/s]



Epoch 19 | Loss 1.3604 || Train F1 0.7060 Acc 0.9264 LieF1 0.4515 || Val F1 0.5270 Acc 0.8962 LieF1 0.1091 || 


Epoch 20: 100%|██████████| 184/184 [00:02<00:00, 62.82it/s]



Epoch 20 | Loss 1.1790 || Train F1 0.7244 Acc 0.9479 LieF1 0.4763 || Val F1 0.5254 Acc 0.9181 LieF1 0.0938 || 


In [29]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

test_ds =DiplomacyGraphDataset("test_with_history_10.jsonl")
test_loader =DataLoader(test_ds,batch_size=1)

model.load_state_dict(torch.load("GATmodel.pt"))
model.eval()

@torch.no_grad()
def get_test_predictions_with_meta(loader):
    all_preds = []
    all_labels = []
    misclassified_0 = []  
    misclassified_1 = []  

    for i, data in enumerate(loader):
        data = data.to(device)
        if data.edge_label.numel() == 0:
            continue

        probs = torch.sigmoid(model(data))
        pred = (probs > 0.5).long().cpu().tolist()
        label = data.edge_label.long().cpu().tolist()

        all_preds.extend(pred)
        all_labels.extend(label)

        idx = 0
        raw_game = test_ds[i] 
        with open("test_with_history_10.jsonl") as f:
            for j, line in enumerate(f):
                if j == i:
                    raw_game_data = json.loads(line)
                    break

        for true, guess in zip(label, pred):
            if true != guess:
                speaker = raw_game_data["speakers"][idx]
                receiver = raw_game_data["receivers"][idx]
                message = raw_game_data["messages"][idx]
                if true == 0 and len(misclassified_0) < 5:
                    misclassified_0.append((speaker, receiver, message))
                elif true == 1 and len(misclassified_1) < 5:
                    misclassified_1.append((speaker, receiver, message))
            idx += 1

    return all_labels, all_preds, misclassified_0, misclassified_1

# Run evaluation
y_true, y_pred, mis_0, mis_1 = get_test_predictions_with_meta(test_loader)

print("\n--- Classification Report (Test Set) ---")
print(classification_report(y_true, y_pred, target_names=["Truthful", "Lie"], digits=3))

print("--- Confusion Matrix (Test Set) ---")
print(confusion_matrix(y_true, y_pred))

lie_precision = precision_score(y_true, y_pred, pos_label=1)
lie_recall    = recall_score(y_true, y_pred, pos_label=1)
lie_f1        = f1_score(y_true, y_pred, pos_label=1)

print(f"\n--- Lie Class Metrics ---")
print(f"Lie Precision: {lie_precision:.4f}")
print(f"Lie Recall:    {lie_recall:.4f}")
print(f"Lie F1 Score:  {lie_f1:.4f}")

print("\n--- 5 Misclassified Truthful Messages (Predicted Lie) ---")
for s, r, m in mis_0:
    print(f"Speaker: {s} | Receiver: {r} | Message: {m}")

print("\n--- 5 Misclassified Lie Messages (Predicted Truthful) ---")
for s, r, m in mis_1:
    print(f"Speaker: {s} | Receiver: {r} | Message: {m}")


/tmp/ipykernel_31/3049420735.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("GATmodel.pt"))



--- Classification Report (Test Set) ---
              precision    recall  f1-score   support

    Truthful      0.936     0.854     0.893      2501
         Lie      0.203     0.388     0.266       240

    accuracy                          0.813      2741
   macro avg      0.569     0.621     0.580      2741
weighted avg      0.871     0.813     0.838      2741

--- Confusion Matrix (Test Set) ---
[[2136  365]
 [ 147   93]]

--- Lie Class Metrics ---
Lie Precision: 0.2031
Lie Recall:    0.3875
Lie F1 Score:  0.2665

--- 5 Misclassified Truthful Messages (Predicted Lie) ---
Speaker: france | Receiver: italy | Message: im sure you expect that im coming to attack you and indeed turkey is trying to set me up against you but the way i look at it is that if we were to split your cp turkey would still win i cant let that happen
Speaker: england | Receiver: germany | Message: i like to work with people so im open to that plan i figured i would probably stalemate with russia for a while whi

In [24]:
from collections import defaultdict

# -------------------------------
# Speaker-wise Error Rates
# -------------------------------
@torch.no_grad()
def get_predictions_with_speaker_meta(loader, raw_jsonl_path):
    preds, labels, speakers = [], [], []

    # Load raw game lines for speaker metadata
    with open(raw_jsonl_path, 'r') as f:
        raw_lines = [json.loads(l) for l in f if l.strip()]

    for i, data in enumerate(loader):
        data = data.to(device)
        if data.edge_label.numel() == 0:
            continue

        prob = torch.sigmoid(model(data))
        pred = (prob > 0.5).long().cpu().tolist()
        label = data.edge_label.long().cpu().tolist()

        preds.extend(pred)
        labels.extend(label)

        # Extract speaker list from the original game line
        game = raw_lines[i]
        speakers.extend(game["speakers"])

    return labels, preds, speakers

# Run inference
y_true, y_pred, speaker_list = get_predictions_with_speaker_meta(test_loader, "test_with_history_10.jsonl")

# Calculate speaker-wise error rates
speaker_stats = defaultdict(lambda: {"correct": 0, "total": 0})

for true, pred, speaker in zip(y_true, y_pred, speaker_list):
    speaker = speaker.lower()
    speaker_stats[speaker]["total"] += 1
    if true == pred:
        speaker_stats[speaker]["correct"] += 1

print("\n--- Speaker-wise Error Rates (Test Set) ---")
for speaker in sorted(speaker_stats):
    correct = speaker_stats[speaker]["correct"]
    total = speaker_stats[speaker]["total"]
    error_rate = 1 - (correct / total) if total > 0 else 0.0
    print(f"{speaker:10s} | Total: {total:3d} | Errors: {total - correct:3d} | Error Rate: {error_rate:.4f}")



--- Speaker-wise Error Rates (Test Set) ---
austria    | Total: 107 | Errors:   8 | Error Rate: 0.0748
england    | Total: 307 | Errors:  42 | Error Rate: 0.1368
france     | Total: 804 | Errors: 159 | Error Rate: 0.1978
germany    | Total: 133 | Errors:  43 | Error Rate: 0.3233
italy      | Total:  97 | Errors:   5 | Error Rate: 0.0515
russia     | Total: 852 | Errors: 169 | Error Rate: 0.1984
turkey     | Total: 441 | Errors:  86 | Error Rate: 0.1950
